In [1]:
import pandas as pd
import numpy as np
from os import path

In [3]:
pip install pymannkendall

Note: you may need to restart the kernel to use updated packages.


In [4]:
from scipy.stats import mannwhitneyu, ttest_ind, bartlett, levene 
from scipy.stats import kendalltau,ansari
from scipy.stats import f as fisher
from sklearn.linear_model import LinearRegression
import pymannkendall as mkm

In [6]:
dr=r"C:\google\2021fall\гидрогео"
fn=r"Severnaya_Dvina_vosstanov_vse_05_11_21.xlsx"
fr=pd.read_excel("Severnaya_Dvina_vosstanov_vse_05_11_21.xlsx",sheet_name=None,index_col=0)
sn=list(fr.keys())
sn

['год', 'Меж', 'МинЗим', 'МинЛ-О']

In [7]:
def mw(x,yr):
    u, p=mannwhitneyu(x[:yr].dropna(),x[yr:].dropna(),alternative="two-sided")
    return p

In [8]:
def tt(x,yr):
    u, p=ttest_ind(x[:yr].dropna(),x[yr:].dropna())
    return [p,x[:yr].mean(),x[yr:].mean(),(x[yr:].mean()-x[:yr].mean())*100/x[:yr].mean()]

In [9]:
def br(x,yr):
    stat, p=bartlett(x[:yr].dropna(),x[yr:].dropna())
    return [p,x[:yr].std(),x[yr:].std(),(x[yr:].std()-x[:yr].std())*100/x[:yr].std()]

In [10]:
def lv(x,yr):
    stat, p=levene(x[:yr].dropna(),x[yr:].dropna())
    return p

In [11]:
def f_test(x, yr):
    a = x[:yr].dropna()
    b = x[yr:].dropna()
    f = b.var(ddof=1)/a.var(ddof=1) #calculate F test statistic
    dfd = a.size-1 #define degrees of freedom numerator 
    dfn = b.size-1 #define degrees of freedom denominator 
    p = 1-fisher.cdf(f, dfn, dfd) #find p-value of F test statistic 
    return p

In [12]:
def an(x,yr):
    stat, p=ansari(x[:yr].dropna(),x[yr:].dropna())
    return p

In [13]:
def mk(x):
    y=x.dropna()
    c, p=kendalltau(y.index,y)
    return p

In [14]:
def mk2(x):
    #y=x.dropna()
    res=mkm.yue_wang_modification_test(x,lag=1)
    return res.p

In [15]:
def lr(x):
    y=x.dropna()
    my_reg=LinearRegression().fit(y.index.to_frame(), y)
    cf1=my_reg.coef_[0]
    return cf1*1000/y.mean()

In [16]:
cl=["p_val tt","Q1","Q2","\u0394Q, %","p_val mw","p_val br","stdQ1",
    "stdQ2","\u0394stdQ, %","p_val lv","f_test p_val","ansari p_val",
    "p_val mk","p_val mk2", "trend %/10 yy"]

In [19]:
fn=path.join(dr, r"stat ND.xlsx")
writer = pd.ExcelWriter("stat ND.xlsx", engine='xlsxwriter')
for i in sn:
    print(i)
    y=1978-1946
    fr2=fr[i].dropna(axis=1,thresh=60)#выбираем район
    res=[pd.DataFrame(fr2.apply(tt,axis=0, yr=y).to_list(), index=fr2.columns),
         fr2.apply(mw,axis=0, yr=y),
         pd.DataFrame(fr2.apply(br,axis=0, yr=y).to_list(), index=fr2.columns),
         fr2.apply(lv,axis=0, yr=y),
         fr2.apply(f_test,axis=0, yr=y),
         fr2.apply(an,axis=0, yr=y),
         fr2.apply(mk,axis=0),fr2.apply(mk2,axis=0),fr2.apply(lr,axis=0)]
    res=pd.concat(res, axis=1)
    res.columns=cl
    res.to_excel(writer,i)
writer.save()

год


AttributeError: 'DataFrame' object has no attribute 'to_list'